# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [18]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [19]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=postgres password=postgres")
cur = conn.cursor()

In [20]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [21]:
song_files = get_files("data/song_data")
print(song_files)

['C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\\udacity\\DataEngineering\\project1\\project-template\\data\\song_data\\A\\A\\A\\TRAAAAW128F429D538.json', 'C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\\udacity\\DataEngineering\\project1\\project-template\\data\\song_data\\A\\A\\A\\TRAAABD128F429CF47.json', 'C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\\udacity\\DataEngineering\\project1\\project-template\\data\\song_data\\A\\A\\A\\TRAAADZ128F9348C2E.json', 'C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\\udacity\\DataEngineering\\project1\\project-template\\data\\song_data\\A\\A\\A\\TRAAAEF128F4273421.json', 'C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\\udacity\\DataEngineering\\project1\\project-template\\data\\song_data\\A\\A\\A\\TRAAAFD128F92F423A.json', 'C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\\udacity\\DataEngineering\\project1\\project-template\\data\\song_data\\A\\A\\A\\TRAAAMO128F1481E7F.json', 'C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\

In [22]:
filepath =  song_files[1]
print(filepath)

C:\Users\sthoe09258\OneDrive - Elekta\Desktop\udacity\DataEngineering\project1\project-template\data\song_data\A\A\A\TRAAABD128F429CF47.json


In [23]:
df = pd.read_json(filepath, typ='series')
df

num_songs                            1
artist_id           ARMJAGH1187FB546F3
artist_latitude               35.14968
artist_longitude             -90.04892
artist_location            Memphis, TN
artist_name               The Box Tops
song_id             SOCIWDW12A8C13D406
title                        Soul Deep
duration                     148.03546
year                              1969
dtype: object

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df
#song_data = list(song_data)[:1]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
song_table_insert = """INSERT INTO SONGS(song_id,title,artist_id, year,duration ) VALUES(%(song_id)s, %(title)s,%(artist_id)s, %(year)s,%(duration)s)"""
print(song_table_insert)
cur.execute(song_table_insert, song_data)
conn.commit()

INSERT INTO SONGS(song_id,title,artist_id, year,duration ) VALUES(%(song_id)s, %(title)s,%(artist_id)s, %(year)s,%(duration)s)


UniqueViolation: duplicate key value violates unique constraint "songs_pkey"
DETAIL:  Key (song_id)=(SOCIWDW12A8C13D406) already exists.


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [24]:
artist_data = df
#artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [25]:
artist_table_insert = """INSERT INTO ARTISTS (artist_id ,name ,location , latitude ,longitude  ) VALUES(%(artist_id)s, %(artist_name)s,%(artist_location)s, %(artist_latitude)s,%(artist_longitude)s)"""
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [30]:
log_files = get_files("data/log_data")
log_files

['C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\\udacity\\DataEngineering\\project1\\project-template\\data\\log_data\\2018\\11\\2018-11-01-events.json',
 'C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\\udacity\\DataEngineering\\project1\\project-template\\data\\log_data\\2018\\11\\2018-11-02-events.json',
 'C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\\udacity\\DataEngineering\\project1\\project-template\\data\\log_data\\2018\\11\\2018-11-03-events.json',
 'C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\\udacity\\DataEngineering\\project1\\project-template\\data\\log_data\\2018\\11\\2018-11-04-events.json',
 'C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\\udacity\\DataEngineering\\project1\\project-template\\data\\log_data\\2018\\11\\2018-11-05-events.json',
 'C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\\udacity\\DataEngineering\\project1\\project-template\\data\\log_data\\2018\\11\\2018-11-06-events.json',
 'C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\

In [31]:
filepath = log_files[0]
filepath

'C:\\Users\\sthoe09258\\OneDrive - Elekta\\Desktop\\udacity\\DataEngineering\\project1\\project-template\\data\\log_data\\2018\\11\\2018-11-01-events.json'

In [34]:
df = pd.read_json(filepath, lines=True ,typ='series')
df.head()
df[0]

{'artist': None,
 'auth': 'Logged In',
 'firstName': 'Walter',
 'gender': 'M',
 'itemInSession': 0,
 'lastName': 'Frye',
 'length': None,
 'level': 'free',
 'location': 'San Francisco-Oakland-Hayward, CA',
 'method': 'GET',
 'page': 'Home',
 'registration': 1540919166796.0,
 'sessionId': 38,
 'song': None,
 'status': 200,
 'ts': 1541105830796,
 'userAgent': '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"',
 'userId': '39'}

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df = 
df.head()

In [ ]:
t = 
t.head()

In [ ]:
time_data = ()
column_labels = ()

In [ ]:
time_df = 
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = 

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.